In [1]:
# Import NumPy for some statistics
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../Dataset/final.csv")

In [3]:
df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,col_50
0,4077.0,3801.0,3588.0,3341.0,3509.0,9578.0,10052.0,10280.0,8968.0,8832.0,...,4815.0,2918.0,4605.0,12363.0,5228.0,4855.0,5013.0,5556.0,6514.0,225
1,3400.0,3065.0,1710.0,3169.0,7538.0,7435.0,7382.0,7149.0,6712.0,6667.0,...,4802.0,2935.0,4860.0,11249.0,4058.0,3586.0,1835.0,4208.0,9019.0,225
2,4518.0,3882.0,3839.0,3623.0,3788.0,6678.0,5938.0,6144.0,5540.0,5463.0,...,9226.0,8795.0,8145.0,8119.0,4128.0,3423.0,1559.0,3836.0,12437.0,225
3,4078.0,3398.0,3342.0,2988.0,3025.0,5294.0,4518.0,3827.0,4085.0,4035.0,...,5541.0,3926.0,5093.0,8735.0,4712.0,4614.0,2746.0,4830.0,8588.0,225
4,4403.0,3869.0,3487.0,3553.0,4081.0,6378.0,5924.0,6098.0,5600.0,5386.0,...,8122.0,7147.0,6846.0,7443.0,4279.0,3817.0,2050.0,4205.0,12931.0,225


In [4]:
labels = df.iloc[:,-1]

In [5]:
data = df.drop(df.columns[50],axis=1)

In [6]:
data.shape

(500000, 50)

In [7]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, MaxPooling1D, Flatten

/home/monster/anaconda3/envs/geo_py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/monster/anaconda3/envs/geo_py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/monster/anaconda3/envs/geo_py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/monster/anaconda3/envs/geo_py37

In [8]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = keras.utils.to_categorical(encoded_Y,num_classes=5) #np_utils.to_categorical(encoded_Y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(data, dummy_y, test_size=0.05, random_state=42,shuffle = True)

In [12]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [13]:
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

In [14]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras import regularizers
from keras import initializers

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)



model = Sequential()
model.add(Dense(200, input_shape=(50, ), activation='relu',kernel_regularizer=regularizers.l2(1e-5),kernel_initializer=keras.initializers.glorot_normal(seed=seed),bias_initializer='zeros'))
model.add(Dropout(5))
model.add(Dense(5, activation='softmax'))
model.summary()

# Compile model
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               10200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1005      
Total params: 11,205
Trainable params: 11,205
Non-trainable params: 0
_________________________________________________________________


In [15]:
sgd = keras.optimizers.Adadelta()
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['acc'])

In [16]:
model.fit(X_train, y_train,
          epochs=10,
          batch_size=100,
         shuffle=True)
score = model.evaluate(X_test, y_test, batch_size=100)
print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))


Epoch 1/10
475000/475000 [==============================] - 4s 8us/step - loss: 0.2349 - acc: 0.9170
Epoch 2/10
475000/475000 [==============================] - 4s 8us/step - loss: 0.1768 - acc: 0.9364
Epoch 3/10
475000/475000 [==============================] - 4s 8us/step - loss: 0.1667 - acc: 0.9398
Epoch 4/10
475000/475000 [==============================] - 4s 8us/step - loss: 0.1603 - acc: 0.9423
Epoch 5/10
475000/475000 [==============================] - 4s 8us/step - loss: 0.1559 - acc: 0.9440
Epoch 6/10
475000/475000 [==============================] - 4s 8us/step - loss: 0.1522 - acc: 0.9455
Epoch 7/10
475000/475000 [==============================] - 4s 8us/step - loss: 0.1496 - acc: 0.9464
Epoch 8/10
475000/475000 [==============================] - 4s 8us/step - loss: 0.1471 - acc: 0.9472
Epoch 9/10
475000/475000 [==============================] - 4s 8us/step - loss: 0.1450 - acc: 0.9481
Epoch 10/10
25000/25000 [==============================] - 0s 4us/step

acc: 94.45%


In [17]:
n_classes = 5

model = Sequential()
model.add(Conv1D(kernel_size = 5, strides=1,filters = 32, activation='relu',input_shape=(50,1)))
                    
print(model.input_shape)
print(model.output_shape)

model.add(MaxPooling1D(pool_size = (2), strides=(2)))
print(model.output_shape)

model.add(Conv1D (kernel_size = 5, strides=1, filters = 64, activation='relu'))
print(model.output_shape)

model.add(MaxPooling1D(pool_size = (2), strides=(2)))
print(model.output_shape)

model.add(Flatten())

print(model.output_shape)

model.add(Dense (1000, activation='relu'))
print(model.output_shape)

model.add(Dense(n_classes, activation = 'softmax'))#,activity_regularizer=keras.regularizers.l2()))
print(model.output_shape)

#model.compile( loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.01), metrics=[keras.metrics.categorical_accuracy])
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])

(None, 50, 1)
(None, 46, 32)

(None, 23, 32)
(None, 19, 64)
(None, 9, 64)
(None, 576)
(None, 1000)
(None, 5)


In [18]:
X1 = np.expand_dims(X_train, axis=2)

In [19]:
X1.shape,y_train.shape

((475000, 50, 1), (475000, 5))

In [20]:
model.fit(X1, y_train, epochs=10, batch_size=100)

Epoch 1/10
475000/475000 [==============================] - 27s 57us/step - loss: 0.2011 - accuracy: 0.9261
Epoch 2/10
475000/475000 [==============================] - 26s 56us/step - loss: 0.1517 - accuracy: 0.9442
Epoch 3/10
475000/475000 [==============================] - 27s 56us/step - loss: 0.1372 - accuracy: 0.9495
Epoch 4/10
475000/475000 [==============================] - 27s 56us/step - loss: 0.1286 - accuracy: 0.9524
Epoch 5/10
475000/475000 [==============================] - 27s 56us/step - loss: 0.1209 - accuracy: 0.9554
Epoch 6/10
475000/475000 [==============================] - 26s 55us/step - loss: 0.1138 - accuracy: 0.9577
Epoch 7/10
475000/475000 [==============================] - 26s 56us/step - loss: 0.1064 - accuracy: 0.9607
Epoch 8/10
475000/475000 [==============================] - 26s 55us/step - loss: 0.0988 - accuracy: 0.9633
Epoch 9/10
475000/475000 [==============================] - 26s 56us/step - loss: 0.0899 - accuracy: 0.9665
Epoch 10/10
475000/475000 [=

In [21]:
X2 = np.expand_dims(X_test, axis=2)

In [22]:
scores = model.evaluate(X2, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

25000/25000 [==============================] - 1s 41us/step

accuracy: 94.98%
